# Assistant to update the poses of an existing scene

Add the objects to track to parameter `/optitrack/objects`

Start the optitrack_publisher:
`roslaunch optitrack_publisher optitrack_publisher.launch ip:=<VRPN_IP>`

In [1]:
import rospy
import rospkg
import json
import yaml
from os.path import join
from tf import TransformListener

In [2]:
rospy.init_node('update_scene_assistant')

In [3]:
rospack = rospkg.RosPack()
tfl = TransformListener()
world = 'base'

In [4]:
with open(rospack.get_path('thr_scenes') + '/config/scenes.yaml') as f:
    scenes = yaml.load(f)

In [5]:
scene = raw_input('Type the name of the scene to update among {}: '.format([scene for scene in scenes]))
assert scene in scenes

Type the name of the scene to update among ['pobax', 'toolbox', 'romeo']: pobax


In [6]:
objects = scenes[scene]
print('Objects of scene {} are:\n {}'.format(scene, objects))

Objects of scene pobax are:
 ['/culbuto/1', '/culbuto/2', '/table']


In [7]:
rospy.set_param('/optitrack/objects', objects)

---> Now run the publisher: roslaunch optitrack_publisher optitrack_publisher.launch <---

In [8]:
with open(join(rospack.get_path('thr_scenes'), 'config', scene, 'poses.json')) as f:
    poses = json.load(f)

In [9]:
def update_action(poses, obj):
    for action in poses:
        if action == 'grasp':
            for pose_index, pose in enumerate(poses[action]):
                raw_input("Please go to GRASP approach pose #{}/{} and press <enter>".format(pose_index + 1, len(poses[action])))
                constraint = tfl.lookupTransform(obj, "right_gripper", rospy.Time(0))
                pose["approach"] = constraint
                raw_input("Please go to GRASP contact pose #{}/{} and press <enter>".format(pose_index + 1, len(poses[action])))
                constraint = tfl.lookupTransform(obj, "right_gripper", rospy.Time(0))
                pose["grasp"] = constraint[0]
        elif action == 'bring':
            raw_input("Please go to bring pose wrt wrist and press <enter>")
            constraint = tfl.lookupTransform("/human/wrist", obj, rospy.Time(0))
            poses[action]["/human/wrist"] = constraint
        elif action == 'place':
            for slave in poses[action]:
                raise NotImplementedError("TODO one via for every slave")
                if slave == 'via':
                    pass
                else:
                    pass
        elif action == 'pick':
            for pose_index, pose in enumerate(poses[action]):
                raw_input("Please go to PICK approach pose #{}/{} and press <enter>".format(pose_index + 1, len(poses[action])))
                constraint = tfl.lookupTransform(obj, "left_gripper", rospy.Time(0))
                pose["approach"] = constraint
                raw_input("Please go to PICK contact pose #{}/{} and press <enter>".format(pose_index + 1, len(poses[action])))
                constraint = tfl.lookupTransform(obj, "left_gripper", rospy.Time(0))
                pose["contact"] = constraint[0]
        elif action == 'give':
            raw_input("Please go to GIVE pose wrt wrist and press <enter>")
            constraint = tfl.lookupTransform("/human/wrist", obj, rospy.Time(0))
            poses[action]["/human/wrist"] = constraint
        elif action == 'hold':
            for pose_index, pose in enumerate(poses[action]):
                raw_input("Please go to HOLD approach pose #{}/{} and press <enter>".format(pose_index + 1, len(poses[action])))
                constraint = tfl.lookupTransform(obj, "right_gripper", rospy.Time(0))
                pose["approach"] = constraint
                raw_input("Please go to HOLD contact pose #{}/{} and press <enter>".format(pose_index + 1, len(poses[action])))
                constraint = tfl.lookupTransform(obj, "right_gripper", rospy.Time(0))
                pose["contact"] = constraint[0]
        elif action == 'constraints':
            for pose_index, pose_dict in enumerate(poses[action]):
                for slave in pose_dict:
                    raw_input("Please place slave {} wrt master {} in pose #{}/{} and press <enter>".format(slave, obj,
                                                                                                            pose_index + 1, len(poses[action])))
                    constraint = tfl.lookupTransform(obj, slave, rospy.Time(0))
                    pose_dict[slave] = constraint
        else:
            raise NotImplementedError("Sorry, the assistant does not know action {} yet".format(action))

In [11]:
answer = ''
for obj in poses:
    if answer != 'q':
        answer = raw_input('Do you want to update object {}? [y/[n]/q] '.format(obj)).lower()
    if answer == 'q':
        break
    elif answer == 'y':
        update_action(poses[obj], obj)
    else:
        print "Warning, you might have issues if {} is also the slave of another object".format(obj)
    print "------------------------------------------"

Do you want to update object /culbuto/2? [y/[n]/q] y
Please go to GRASP approach pose #1/1 and press <enter>
Please go to GRASP contact pose #1/1 and press <enter>
------------------------------------------
Do you want to update object /culbuto/1? [y/[n]/q] y
Please go to GRASP approach pose #1/1 and press <enter>
Please go to GRASP contact pose #1/1 and press <enter>
------------------------------------------
Do you want to update object /table? [y/[n]/q] y


NotImplementedError: TODO one via for every slave

In [12]:
with open(join(rospack.get_path('thr_scenes'), 'config', scene, 'poses.json'), 'w') as f:
    json.dump(poses, f, indent=4)